In [2]:
import numpy as np

In [3]:
limit = 10000
offset = 0

def get_firms_ids(cursor):
  query = f"select id from firms offset {str(offset)} limit {str(limit)}"
  cursor.execute(query)
  return cursor.fetchall()

def compute_geo_firms(id):
  query = """
    with
    envelope as (
      select ST_MakeEnvelope(
        ST_X(ST_Project(firms.point, 1000, radians(90))::geometry), 
        ST_Y(ST_Project(firms.point, 1000, radians(180))::geometry), 
        ST_X(ST_Project(firms.point, 1000, radians(-90))::geometry),
        ST_Y(ST_Project(firms.point, 1000, radians(0))::geometry),
        4326
      ) as geom, firms.point as point, date, brightness, brightness_t31, radiative_power
      from firms where id = %(id)s
    ),
    roads as (
      select ST_Union(ST_Intersection(envelope.geom, ukraine_roads.geom)) as geom
      from ukraine_roads, envelope
      where ST_Intersects(envelope.geom, ukraine_roads.geom)
    ),
    rails as (
      select ST_Union(ST_Intersection(envelope.geom, ukraine_railways.geom)) as geom
      from ukraine_railways, envelope
      where ST_Intersects(envelope.geom, ukraine_railways.geom)
    ),
    settlements as (
      select population, ukraine_settlements.geom as geom, name,
      ST_Distance(ukraine_settlements.geom::geography, envelope.point::geography) as distance
      from ukraine_settlements, envelope
      where population > 0
      order by distance asc
      limit 10
    ),
    center as (
      select ARRAY_AGG(population) as population, ARRAY_AGG(distance) as distance from settlements
    )
    select 
    	ST_X(envelope.point) as lon, ST_Y(envelope.point) as lat,
      date, brightness, brightness_t31, radiative_power,
      ST_Distance(roads.geom::geography, envelope.point::geography) as dist_road,
      ST_Distance(rails.geom::geography, envelope.point::geography) as dist_rail,
      population, distance
    from roads, rails, envelope, center
  """
  def execute(cursor):
    cursor.execute(query, { "id": id })
    return cursor.fetchall()
    
  return execute

def insert_geo_firms(geo):
  query = """
    insert into geo_firms values (%s, ST_Point(%s, %s, 4326), %s, %s, %s, %s, %s, %s, %s)
  """

  def execute(cursor):
    cursor.execute(query, geo)
    
  return execute

In [12]:
from DBEngine import db_engine

firms = db_engine.execute(get_firms_ids)
for entry in firms:
  geo = db_engine.execute(compute_geo_firms(entry[0]))[0]
  geo = (entry[0],) + geo[:8] + (np.mean(np.array(geo[8]) / ((np.array(geo[9]) ** (1/2)) + 1)),)
  db_engine.execute(insert_geo_firms(geo))

db_engine.commit()

In [11]:
offset += limit
limit = 10000
print(offset, limit)

130000 10000


In [ ]:
db_engine.rollback()